<a href="https://colab.research.google.com/github/rajkumarin/Deep_Learning_on_Images/blob/main/ANLY_535_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **STEP1. Introduction to GPU**

In [1]:
from tensorflow.python.client import device_lib
print("Show System RAM Memory:\n\n")
!cat /proc/meminfo | egrep "MemTotal*"
print("\n\nShow Devices:\n\n"+str(device_lib.list_local_devices()))

Show System RAM Memory:


MemTotal:       13305368 kB


Show Devices:

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8880522304696618421
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14509932544
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2107300744349938812
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


In [2]:
import tensorflow as tf
tf.__version__

'2.5.0'

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import time
start = time.time()
!python3 "/content/gdrive/MyDrive/Colab Notebooks/mnist_cnn.py"
end = time.time()
print(end - start)

2021-07-01 15:43:55.631329: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
2021-07-01 15:43:57.748668: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-01 15:43:57.753756: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-07-01 15:43:57.754317: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-07-01 15:43:57.754359: I tensorflow/stream_executor/platform/default/dso_loa

**Q1- A Keras code is provided for running hand written recognition on both GPU and CPU. Run the code on colab and your own machine and compare the results.**

 - It took 89 seconds using a GPU while a CPU takes xxxxxx seconds

# **STEP2. Implement handwritten recognition in Tensorflow using CNN**

In [5]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
batch_size = 128
num_classes = 10
epochs = 12
# input image dimensions
img_rows, img_cols = 28, 28
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()


In [6]:
if K.image_data_format() == 'channels_first':
 x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
 x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
 input_shape = (1, img_rows, img_cols)
else:
 x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
 x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
 input_shape = (img_rows, img_cols, 1)

In [7]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [8]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',
input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

**Q2- Explain the way that this model is designed. Talk about all the layers and their functionality.**

- The model is a CNN
- It has 2 convolutional layers
- A pooling layer with 25% dropout
- A flat NN with one hidden layer with 50% dropout and a output layer

In [9]:
model.compile(loss='categorical_crossentropy',
 optimizer='adam',
 metrics=['accuracy'])
 
model.fit(x_train, y_train,
 batch_size=batch_size,
 epochs=epochs,
 verbose=1,
 validation_split=0.2)
 
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Epoch 1/12
375/375 [==============================] - 6s 10ms/step - loss: 0.2961 - accuracy: 0.9066 - val_loss: 0.0680 - val_accuracy: 0.9808
Epoch 2/12
375/375 [==============================] - 3s 8ms/step - loss: 0.1015 - accuracy: 0.9698 - val_loss: 0.0514 - val_accuracy: 0.9849
Epoch 3/12
375/375 [==============================] - 3s 8ms/step - loss: 0.0700 - accuracy: 0.9786 - val_loss: 0.0433 - val_accuracy: 0.9874
Epoch 4/12
375/375 [==============================] - 3s 8ms/step - loss: 0.0577 - accuracy: 0.9828 - val_loss: 0.0433 - val_accuracy: 0.9876
Epoch 5/12
375/375 [==============================] - 3s 8ms/step - loss: 0.0503 - accuracy: 0.9837 - val_loss: 0.0388 - val_accuracy: 0.9893
Epoch 6/12
375/375 [==============================] - 3s 8ms/step - loss: 0.0432 - accuracy: 0.9863 - val_loss: 0.0440 - val_accuracy: 0.9886
Epoch 7/12
375/375 [==============================] - 3s 8ms/step - loss: 0.0385 - accuracy: 0.9879 - val_loss: 0.0388 - val_accuracy: 0.9899
Epoch

**Q3- Design the learning curve and talk about what you see.**

# **STEP3. Text mining using CNN**

## 3.1. Pre-Processing

In [ ]:
import pandas as pd
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/amazon_cells_labelled.txt', names=[
'sentence', 'label'], sep='\t')


In [ ]:
print(df.iloc[0])


sentence    So there is no way for me to plug it in here i...
label                                                       0
Name: 0, dtype: object


**Q4- Explain these graphs. If you see any issue, suggest a solution to resolve it. Make the model by creating 3 hidden layers (first one 200 nodes, second one 100 nodes and last one 50 nodes and after each step, add dropout of 0.2 and report the accuracy.**

- ANS

## 3.2. Embedded word

## 3.3. Model training

**Q5- How do you interpret these results?**

- ANS

**Q6- What is your recommendation to improve the accuracy? Implement your idea.**

- ANS

In [ ]:
%shell jupyter nbconvert --to html '//content/gdrive/My Drive/Colab
Notebooks/filename.ipynb'